In [1]:
#Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

#Import API key
from config import g_key

In [2]:
#Import weather_data.csv from WeatherPY 

weather_data = pd.read_csv('../WeatherPY/weather_data.csv')
weather_data

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rio Grande,-32.0350,-52.0986,300.15,50,20,4.12,BR,1615558960
1,Hermanus,-34.4187,19.2345,296.48,67,100,0.45,ZA,1615558962
2,Qaanaaq,77.4840,-69.3632,257.07,69,100,0.77,GL,1615558964
3,Avera,33.1940,-82.5271,291.48,82,1,2.06,US,1615558965
4,Rikitea,-23.1203,-134.9692,299.29,76,100,3.78,PF,1615558966
...,...,...,...,...,...,...,...,...,...
566,Ormara,25.2088,64.6357,296.94,81,14,1.63,PK,1615559751
567,Price,39.5994,-110.8107,271.15,74,20,5.14,US,1615559752
568,Samarinda,-0.5000,117.1500,296.55,93,89,0.77,ID,1615559756
569,Teya,60.3778,92.6267,261.90,89,62,4.03,RU,1615559757


# Humidity Heatmap

In [3]:
#Configure gmaps.
#Use the Lat and Lng as locations and Humidity as the weight.

#Store 'Lat' and 'Lng' into  locations 
locations = weather_data[["Lat", "Lng"]].astype(float)

#Store humidity into humidity 
humidity = weather_data["Humidity"]

In [5]:
#Add Heatmap layer to map.

#plot heatmap
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
#Create new Dataframe fitting weather criteria
#Narrow down the cities to fit weather conditions.
#Drop any rows will null values.

max_temp_df = weather_data.loc[(weather_data["Max temp"] < 80) & (weather_data["Max temp"] > 70)]
#max_temp_df.head()

#**My max temperature are over 100 degrees so therefore I cannot process this request**

wind_speed_df = weather_data.loc[(weather_data["Wind Speed"] <= 10)]
#wind_speed_df.head()

cloud_data_df = weather_data.loc[(weather_data["Cloudiness"] == 0)]
#cloud_data_df.head()

In [7]:
#Merge the two dataframes together
specific_cities_df = pd.merge(wind_speed_df, cloud_data_df, how="inner", on=None)
specific_cities_df

#Remove rows missing data
specific_cities_df = specific_cities_df.dropna()
specific_cities_df


,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Saint-Philippe,-21.3585,55.7679,300.15,65,0,7.72,RE,1615558972
1,Albany,42.6001,-73.9662,283.15,40,0,1.34,US,1615558803
2,Belvedere Marittimo,39.6179,15.8617,279.15,87,0,2.57,IT,1615558981
3,Ushuaia,-54.8000,-68.3000,285.15,66,0,5.14,AR,1615558863
4,Svetlaya,46.5389,138.3297,270.44,72,0,2.94,RU,1615558990
...,...,...,...,...,...,...,...,...,...
86,Goderich,43.7501,-81.7165,276.15,81,0,5.66,CA,1615559730
87,Vila Velha,-20.3297,-40.2925,303.15,62,0,6.17,BR,1615559508
88,Kidal,18.4411,1.4078,303.28,9,0,8.84,ML,1615559733
89,Karratha,-20.7377,116.8463,302.46,68,0,1.29,AU,1615559748


# Hotel Map

In [8]:
#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.

hotel_df = specific_cities_df.loc[:,["City", "Lat", "Lng", "Max temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint-Philippe,-21.3585,55.7679,300.15,65,0,7.72,RE,1615558972,
1,Albany,42.6001,-73.9662,283.15,40,0,1.34,US,1615558803,
2,Belvedere Marittimo,39.6179,15.8617,279.15,87,0,2.57,IT,1615558981,
3,Ushuaia,-54.8000,-68.3000,285.15,66,0,5.14,AR,1615558863,
4,Svetlaya,46.5389,138.3297,270.44,72,0,2.94,RU,1615558990,
...,...,...,...,...,...,...,...,...,...,...
86,Goderich,43.7501,-81.7165,276.15,81,0,5.66,CA,1615559730,
87,Vila Velha,-20.3297,-40.2925,303.15,62,0,6.17,BR,1615559508,
88,Kidal,18.4411,1.4078,303.28,9,0,8.84,ML,1615559733,
89,Karratha,-20.7377,116.8463,302.46,68,0,1.29,AU,1615559748,


In [9]:
#Set parameters to search for hotels with 5000 meters.
#Hit the Google Places API for each city's coordinates.

#Set up url & lists to hold response info
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
my_params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
     
        lat = row["Lat"]
        lng = row["Lng"]
        city_name = row["City"]
        
        my_params['location'] = f"{lat},{lng}"
        
        print(f"Retrieving Results for Index {index}: {city_name}.")
        response = requests.get(base_url, params=my_params).json()
        #print(response)
        
        results = response["results"]
        
        try:
            print(f"The closest hotel in {city_name} is {results[0]['name']}.")
            hotel_df.loc[index, "Hotel Name"] = results[0]['name']
            
        except (KeyError, IndexError):
            print("Missing field/result...Skipping.")
            
        print("-----------------------")
        print(f'Data Retrieval Complete')
            
        


Retrieving Results for Index 0: Saint-Philippe.
The closest hotel in Saint-Philippe is Chambres d'hôte "La Trinité".
-----------------------
Data Retrieval Complete
Retrieving Results for Index 1: Albany.
Missing field/result...Skipping.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 2: Belvedere Marittimo.
The closest hotel in Belvedere Marittimo is La Marinella.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 3: Ushuaia.
The closest hotel in Ushuaia is Albatros Hotel.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 4: Svetlaya.
Missing field/result...Skipping.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 5: Busselton.
The closest hotel in Busselton is Observatory Guest House.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 6: Broken Hill.
The closest hotel in Broken Hill is Royal Exchange Hotel.
-----------------------
Data Retrieval Com

The closest hotel in San Vicente is Complejo Turístico Farelluk.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 58: Zunheboto.
The closest hotel in Zunheboto is Inspection Bungalow.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 59: Nemuro.
The closest hotel in Nemuro is Hotel Nemuro Kaiyoutei.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 60: Constitución.
The closest hotel in Constitución is Hotel Boutique Las Azucenas.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 61: Marzuq.
Missing field/result...Skipping.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 62: Konosha.
The closest hotel in Konosha is Gostinitsa Miks.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 63: Tokat.
Missing field/result...Skipping.
-----------------------
Data Retrieval Complete
Retrieving Results for Index 64: Alice Town.
The clos

In [10]:
#Store the first Hotel result into the DataFrame.

hotel_df

,City,Lat,Lng,Max temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Saint-Philippe,-21.3585,55.7679,300.15,65,0,7.72,RE,1615558972,"Chambres d'hôte ""La Trinité"""
1,Albany,42.6001,-73.9662,283.15,40,0,1.34,US,1615558803,
2,Belvedere Marittimo,39.6179,15.8617,279.15,87,0,2.57,IT,1615558981,La Marinella
3,Ushuaia,-54.8000,-68.3000,285.15,66,0,5.14,AR,1615558863,Albatros Hotel
4,Svetlaya,46.5389,138.3297,270.44,72,0,2.94,RU,1615558990,
...,...,...,...,...,...,...,...,...,...,...
86,Goderich,43.7501,-81.7165,276.15,81,0,5.66,CA,1615559730,Harmony Inn
87,Vila Velha,-20.3297,-40.2925,303.15,62,0,6.17,BR,1615559508,Quality Suites Vila Velha
88,Kidal,18.4411,1.4078,303.28,9,0,8.84,ML,1615559733,
89,Karratha,-20.7377,116.8463,302.46,68,0,1.29,AU,1615559748,ibis Styles Karratha


In [11]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content= hotel_info)

# Display figure
fig = gmaps.figure()
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))